<a href="https://colab.research.google.com/github/quazimo1/ANDREW-NG-notes/blob/main/neural_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import torch
import torchvision
import matplotlib.pyplot as plt
from time import time
from torchvision import datasets, transforms
from torch import nn, optim
import pandas as pd
from sklearn.preprocessing import MinMaxScaler    
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
dataset=pd.read_csv('/content/drive/MyDrive/data_set/fashion_neural.csv')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
dependent_variable='label'
independent_variables=dataset.columns.tolist()
independent_variables.remove(dependent_variable)
X=dataset[independent_variables].values
Y=dataset[dependent_variable].values
X_train, X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=0)
scaler=MinMaxScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [ ]:
batch_size = 100

trainset = TensorDataset(torch.FloatTensor(X_train), torch.LongTensor(Y_train))
valset = TensorDataset(torch.FloatTensor(X_test), torch.LongTensor(Y_test))
trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=False)
valloader= DataLoader(valset, batch_size=batch_size, shuffle=False) 


In [ ]:
input_size = 784
hidden_sizes = [200, 100]
output_size = 10

model = nn.Sequential(nn.Linear(input_size, hidden_sizes[0]),
                      nn.ReLU(),
                      nn.Linear(hidden_sizes[0], hidden_sizes[1]),
                      nn.ReLU(),
                      nn.Linear(hidden_sizes[1], output_size),
                      nn.LogSoftmax(dim=1))
print(model)

Sequential(
  (0): Linear(in_features=784, out_features=200, bias=True)
  (1): ReLU()
  (2): Linear(in_features=200, out_features=100, bias=True)
  (3): ReLU()
  (4): Linear(in_features=100, out_features=10, bias=True)
  (5): LogSoftmax(dim=1)
)


In [ ]:
criterion = nn.NLLLoss()
images, labels = next(iter(trainloader))
images = images.view(images.shape[0], -1)

logps = model(images) #log probabilities
loss = criterion(logps, labels) #calculate the NLL loss

In [ ]:
optimizer = optim.SGD(model.parameters(), lr=0.003, momentum=0.9)
time0 = time()
epochs = 20
for e in range(epochs):
    running_loss = 0
    for images, labels in trainloader:
        
        images = images.view(images.shape[0], -1)
    
        optimizer.zero_grad()
        
        output = model(images)
        loss = criterion(output, labels)
      
        loss.backward()
        
        optimizer.step()
        
        running_loss += loss.item()
    else:
        print("Epoch {} - Training loss: {}".format(e, running_loss/len(trainloader)))
print("\nTraining Time (in minutes) =",(time()-time0)/60)

Epoch 0 - Training loss: 2.234891253709793
Epoch 1 - Training loss: 1.828180728852749
Epoch 2 - Training loss: 1.1696679323911667
Epoch 3 - Training loss: 0.9210743129253387
Epoch 4 - Training loss: 0.817867012321949
Epoch 5 - Training loss: 0.7521111972630024
Epoch 6 - Training loss: 0.7036901220679284
Epoch 7 - Training loss: 0.665532848611474
Epoch 8 - Training loss: 0.634427922219038
Epoch 9 - Training loss: 0.6085112553089858
Epoch 10 - Training loss: 0.5864146798849106
Epoch 11 - Training loss: 0.5672990422695875
Epoch 12 - Training loss: 0.5504825465381146
Epoch 13 - Training loss: 0.5355267733335495
Epoch 14 - Training loss: 0.5222433757036924
Epoch 15 - Training loss: 0.5103865262120962
Epoch 16 - Training loss: 0.4997402735054493
Epoch 17 - Training loss: 0.49007458575069907
Epoch 18 - Training loss: 0.4812259439378977
Epoch 19 - Training loss: 0.4730673283338547

Training Time (in minutes) = 0.0731657346089681


In [ ]:
correct_count, all_count = 0, 0
for images,labels in valloader:
  for i in range(len(labels)):
    img = images[i].view(1, 784)
    with torch.no_grad():
        logps = model(img)

    
    ps = torch.exp(logps)
    probab = list(ps.numpy()[0])
    pred_label = probab.index(max(probab))
    true_label = labels.numpy()[i]
    if(true_label == pred_label):
      correct_count += 1
    all_count += 1

print("Accuracy % =", ((correct_count/all_count)*100))

Accuracy % = 82.19999999999999
